# Ejercicios de Web scraping
En esta práctica vamos a realizar distintos ejercicios sobre captura de contenidos web (web scraping) usando las librerías vistas en clase (`request` y `BeautifulSoup`).

In [1]:
import requests
from bs4 import BeautifulSoup

## Parte 1: Captura de datos meteorológicos
En este parte vamos a capturar datos meteorológicos de la Comunitat Valenciana desde la página de la [AVAMET (Associació valenciana de meteorologia)](https://www.avamet.org) con request and BeautifulSoup.

In [5]:
fecha='2022-03-16'
r=requests.get("https://www.avamet.org/mx-meteoxarxa.php", params={'id':fecha})

In [6]:
soup=BeautifulSoup(r.text, "html.parser")

Los datos de todos los municipios de la CV están en una tabla de clase `tDades`

In [7]:
tabla=soup.find("table", class_="tDades")
tabla

<table class="tDades" style="width: 98% !important;">
<thead class="fixe1">
<tr>
<th rowspan="2" width="35%">Estació</th>
<th colspan="3">Temperatura (°C)</th>
<th>HR (%)</th>
<th rowspan="2" width="10%">Prec. (mm)</th>
<th colspan="3">Vent</th>
</tr>
<tr>
<th width="7%">Mín</th>
<th width="7%">Mit</th>
<th width="7%">Màx</th>
<th width="7%">Mit</th>
<th width="8%">Mit</th>
<th width="8%">Dir</th>
<th width="8%">Màx</th>
</tr>
</thead>
<tr>
<td class="rProvincia" colspan="8">Província de Castelló</td>
</tr>
<tr>
<td class="rComarca" colspan="8">els Ports</td>
</tr>
<tr>
<td class="rEsta"><a class="negre" href="mx-fitxa.php?id=c01m038e20"><img alt="" height="13" src="imatges/2017/clas/estrela-mx-.png" title="" width="13"> Castellfort<span class="rEstaDmxo"><span class="ptda"></span>AEMET</span></img></a> </td>
<td class="rValm colornT17">5,2 </td>
<td class="rValm colornT17">5,8 </td>
<td class="rValm colornT18">6,5 </td>
<td class="rVal"> </td>
<td class="rValm colorP00">0,1 </td>
<td 

Dentro de la tabla, los datos están en las filas (`<tr>`) que tienen un elemento`<td class='rEsta'>`. Definimos una función para filtrar etiquetas con esta clase y buscamos todos los elementos internos a la tabla:

In [8]:
def clase_rEsta(tag):
    return tag.find(class_="rEsta")

In [9]:
loc=tabla.find_all(clase_rEsta)

In [10]:
len(loc)

745

Nos fijamos por ejemplo en el primer elemento de esta lista:

In [11]:
print(loc[0].prettify())

<tr>
 <td class="rEsta">
  <a class="negre" href="mx-fitxa.php?id=c01m038e20">
   <img alt="" height="13" src="imatges/2017/clas/estrela-mx-.png" title="" width="13">
    Castellfort
    <span class="rEstaDmxo">
     <span class="ptda">
     </span>
     AEMET
    </span>
   </img>
  </a>
 </td>
 <td class="rValm colornT17">
  5,2
 </td>
 <td class="rValm colornT17">
  5,8
 </td>
 <td class="rValm colornT18">
  6,5
 </td>
 <td class="rVal">
 </td>
 <td class="rValm colorP00">
  0,1
 </td>
 <td class="rVal">
 </td>
 <td class="rVal">
 </td>
 <td class="rVal">
  <b>
  </b>
 </td>
</tr>



Vemos que algunas de las celdas de esta fila tienen elementos de tipo `<span>`. Si no nos interesan los podríamos eliminar con el método `.decompose()` del Tag. Pero como el texto nos interesa insertamos un espacio para separar el contenido al extraer el texto posteriormente con `.text()`:

In [12]:
for t in loc:
    for b in t.find_all('span', class_="rEstaDmxo"):
        b.insert_before(' ')
print(loc[0].prettify())

<tr>
 <td class="rEsta">
  <a class="negre" href="mx-fitxa.php?id=c01m038e20">
   <img alt="" height="13" src="imatges/2017/clas/estrela-mx-.png" title="" width="13">
    Castellfort
    <span class="rEstaDmxo">
     <span class="ptda">
     </span>
     AEMET
    </span>
   </img>
  </a>
 </td>
 <td class="rValm colornT17">
  5,2
 </td>
 <td class="rValm colornT17">
  5,8
 </td>
 <td class="rValm colornT18">
  6,5
 </td>
 <td class="rVal">
 </td>
 <td class="rValm colorP00">
  0,1
 </td>
 <td class="rVal">
 </td>
 <td class="rVal">
 </td>
 <td class="rVal">
  <b>
  </b>
 </td>
</tr>



In [13]:
[t.text.strip() for t in loc[0].find_all("td")]

['Castellfort AEMET', '5,2', '5,8', '6,5', '', '0,1', '', '', '']

In [16]:
[t for t in loc[0].find_all("td")]

[<td class="rEsta"><a class="negre" href="mx-fitxa.php?id=c01m038e20"><img alt="" height="13" src="imatges/2017/clas/estrela-mx-.png" title="" width="13"> Castellfort <span class="rEstaDmxo"><span class="ptda"></span>AEMET</span></img></a> </td>,
 <td class="rValm colornT17">5,2 </td>,
 <td class="rValm colornT17">5,8 </td>,
 <td class="rValm colornT18">6,5 </td>,
 <td class="rVal"> </td>,
 <td class="rValm colorP00">0,1 </td>,
 <td class="rVal"> </td>,
 <td class="rVal"> </td>,
 <td class="rVal"><b> </b></td>]

In [17]:
datos = [[t.text.strip() for t in fila.find_all("td")] for fila in loc]

In [18]:
len(datos[0])

9

Ahora exportamos la tabla como un DataFrame de `pandas`:

In [19]:
import pandas as pd

In [20]:
data_matrix = pd.DataFrame(datos)

In [21]:
data_matrix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 745 entries, 0 to 744
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       745 non-null    object
 1   1       745 non-null    object
 2   2       745 non-null    object
 3   3       745 non-null    object
 4   4       745 non-null    object
 5   5       745 non-null    object
 6   6       745 non-null    object
 7   7       745 non-null    object
 8   8       745 non-null    object
dtypes: object(9)
memory usage: 52.5+ KB


In [24]:
data_matrix.columns = ['localidad', 'Temp', 'Tmax', 'Tmin', 'Humedad', 'Precip', 'Vel.viento', 'Dir.viento', 'Vmax_viento']

,localidad,Temp,Tmax,Tmin,Humedad,Precip,Vel.viento,Dir.viento,Vmax_viento
0,Castellfort AEMET,"5,2","5,8","6,5",,"0,1",,,
1,Cinctorres,"7,8","8,3","9,2",91,"1,0","6,4",ESE,"25,7"
2,Forcall,"9,4","10,2","11,2",88,"0,4","5,8",N,"25,7"
3,la Mata,"8,6","9,1","10,3",89,"0,2","5,6",ESE,"37,0"
4,Morella centre,"7,1","8,0","9,2",94,"0,0","5,8",ENE,"33,8"


### Ejercicio 1
Crea un script para capturar los datos de un territorio y una fecha concretas a través de la URL:\
`https://www.avamet.org/mx-meteoxarxa.php?id={fecha}&territori={territorio}`\
Los códigos de cada territorio están en el elemento `select` siguiente:

In [25]:
soup.find("select", attrs={'name': "freg_territori"})

<select class="formBasic" name="freg_territori" onchange="location.href='mx-meteoxarxa.php?id=2022-03-16&amp;territori=' + this.value">
<option selected="" style="font-weight:800;" value="pv">TOT EL TERRITORI</option>
<option style="font-weight:800;" value="p12">Prov. Castelló</option>
<option value="c01">   els Ports</option>
<option value="c02">   l′Alt Maestrat</option>
<option value="c03">   el Baix Maestrat</option>
<option value="c04">   l′Alcalatén</option>
<option value="c05">   la Plana Alta</option>
<option value="c06">   la Plana Baixa</option>
<option value="c07">   l′Alt Palància</option>
<option value="c08">   l′Alt Millars</option>
<option style="font-weight:800;" value="p46">Prov. València</option>
<option value="c09">   el Racó d′Ademús</option>
<option value="c10">   la Serrania</option>
<option value="c11">   el Camp de Túria</option>
<option value="c12">   el Camp de Morvedre</option>
<option value="c13">   l′Horta Nord</option>
<option value="c14">   l′Horta Oest</

In [28]:
def scrape_avamet_data(fecha, territorio):
    url = f"https://www.avamet.org/mx-meteoxarxa.php?id={fecha}&territori={territorio}"
    r = requests.get(url)

    if r.status_code == 200:
        soup = BeautifulSoup(r.text, "html.parser")  # Cambiar "html.parse" por "html.parser"
        tabla=soup.find("table", class_="tDades")  # Buscar la tabla correcta con clase "tDades"

        if tabla:
            # Usar la misma lógica que en el ejemplo inicial
            def clase_rEsta(tag):
                return tag.find(class_="rEsta")
            
            loc = tabla.find_all(clase_rEsta)
            
            # Insertar espacios antes de los spans
            for t in loc:
                for b in t.find_all('span', class_="rEstaDmxo"):
                    b.insert_before(' ')
            
            # Extraer los datos
            datos_estaciones = [[td.text.strip() for td in fila.find_all("td")] for fila in loc]
            
            if datos_estaciones:
                df = pd.DataFrame(datos_estaciones)
                df.columns=['Estacion', 'Temperatura', 'Máxima', 'Mínima', 'Humedad', 'Precipitación', 'Velocidad viento', 'Dirección viento', 'Viento Máximo']
                return df
            else:
                print("No se encontraron datos en la tabla")
                return None
        else:
            print("No se encontró la tabla")
            return None
    else:
        print(f"La solicitud GET falló con código: {r.status_code}")
        return None

In [29]:

fecha='2022-03-16'
territorio='2'
datos=scrape_avamet_data(fecha, territorio)

if datos is not None:
    print(datos.head())
else:
    print("Error")

            Estacion Temperatura Máxima Mínima Humedad Precipitación  \
0  Castellfort AEMET         5,2    5,8    6,5                   0,1   
1         Cinctorres         7,8    8,3    9,2      91           1,0   
2            Forcall         9,4   10,2   11,2      88           0,4   
3            la Mata         8,6    9,1   10,3      89           0,2   
4     Morella centro         7,1    8,0    9,2      94           0,0   

  Velocidad viento Dirección viento Viento Máximo  
0                                                  
1              6,4              ESE          25,7  
2              5,8                N          25,7  
3              5,6              ESE          37,0  
4              5,8              ENE          33,8  


### Ejercicio 2:
Captura los datos durante un mes de la estación de 'València Camins al Grau' y representa gráficamente su temperatura media 

In [31]:
import matplotlib.pyplot as plt
import datetime

In [39]:
def scrape_avamet_monthly_data(month, year, territory, station):
    data_frame=[]

    num_days=(datetime.datetime(year, month+1, 1)-datetime.datetime(year, month, 1)).days
    for day in range(1, num_days+1):
        fecha=f"{year}-{month:02d}-{day:02d}"
        url = f"https://www.avamet.org/mx-meteoxarxa.php?id={fecha}&territori={territory}"
        r = requests.get(url)

        if r.status_code == 200:
            soup = BeautifulSoup(r.text, "html.parser") 
            tabla=soup.find("table", class_="tDades")  

            if tabla:
                rows=tabla.find_all("tr")
                for row in rows:
                    station_name= row.find("td", class_="rEsta")
                    if station_name and station in station_name.text.strip():  
                        data=[elem.text.strip() for elem in row.find_all("td")]
                        if len(data) == 9:
                            data_frame.append(data)
                        break
        else:
            print(f"No se pudo obtener datos para {fecha}")
    
    if data_frame:
        df=pd.DataFrame(data_frame)
        df.columns = ['Estacion', 'Temperatura', 'Máxima', 'Mínima', 'Humedad', 'Precipitación', 'Velocidad viento', 'Dirección viento', 'Viento Máximo']
        return df
    else:
        return None

In [40]:
month = 3
year = 2022
territorio = 'p46'
station_keyword = 'ValènciaCamins al Grau'
data = scrape_avamet_monthly_data(month, year, territorio, station_keyword)

if data is not None:
    # Convertir temperaturas de string a float
    data[['Temperatura', 'Máxima', 'Mínima']] = data[['Temperatura', 'Máxima', 'Mínima']].apply(lambda x: x.str.replace(',', '.').astype(float))

    avg_temp = data['Temperatura'].mean()
    print(f"Temperatura promedio para {station_keyword} en {year}-{month:02d}: {avg_temp:.2f}°C")

    plt.figure(figsize=(10, 6))
    plt.plot(range(1, len(data)+1), data['Temperatura'], marker='o', linestyle='-')
    plt.axhline(y=avg_temp, color='r', linestyle='--', label=f"Temperatura promedio: {avg_temp:.2f}°C")
    plt.xlabel('Día del mes')
    plt.ylabel('Temperatura (°C)')
    plt.title(f'Temperaturas en {station_keyword} - {year}-{month:02d}')
    plt.legend()
    plt.grid(True)
    plt.show()
else:
    print("No hay datos disponibles")

No hay datos disponibles


## Parte 2: Datos de la Wikipedia
En esta parte vamos a obtener las URL de las entradas en la wikipedia para todas las provincias de España y vamos a obtener de ellas sus datos básicos en forma de tabla.\
El listado de las provincias de España se puede descargar de la página de la wikipedia siguiente:\
https://es.wikipedia.org/wiki/Provincia_(España) Este ejercicio también se realizará con request y BeautifulSoup.

In [6]:
import requests
from bs4 import BeautifulSoup

In [7]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
url_provincias = "https://es.wikipedia.org/wiki/Provincia_(España)"
respuesta_provincias = requests.get(url_provincias, headers=headers)
respuesta_provincias.raise_for_status()

In [8]:
sopa_provincias = BeautifulSoup(respuesta_provincias.text, "html.parser")

In [9]:
tabla_provincias = sopa_provincias.find("table", {"class": "wikitable sortable"})
filas_provincias = tabla_provincias.find_all("tr")[1:]
len(filas_provincias)

50

In [11]:
filas_provincias[1].td.find_all("a")

[<a class="mw-file-description" href="/wiki/Archivo:Bandera_provincia_Albacete.svg"><img class="mw-file-element" data-file-height="500" data-file-width="750" decoding="async" height="13" src="//upload.wikimedia.org/wikipedia/commons/thumb/d/d6/Bandera_provincia_Albacete.svg/20px-Bandera_provincia_Albacete.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/d/d6/Bandera_provincia_Albacete.svg/40px-Bandera_provincia_Albacete.svg.png 1.5x" width="20"/></a>,
 <a href="/wiki/Provincia_de_Albacete" title="Provincia de Albacete">Albacete</a>]

In [12]:
enlaces = []
for fila in filas_provincias:
    if fila.td:
        enlaces_fila = fila.td.find_all("a")
        for r in enlaces_fila:
            enlaces.append(r)

In [13]:
enlaces

[<a class="mw-file-description" href="/wiki/Archivo:Flag_of_%C3%81lava.svg"><img class="mw-file-element" data-file-height="500" data-file-width="750" decoding="async" height="13" src="//upload.wikimedia.org/wikipedia/commons/thumb/1/1f/Flag_of_%C3%81lava.svg/20px-Flag_of_%C3%81lava.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/1/1f/Flag_of_%C3%81lava.svg/40px-Flag_of_%C3%81lava.svg.png 1.5x" width="20"/></a>,
 <a href="/wiki/%C3%81lava" title="Álava">Álava</a>,
 <a class="mw-file-description" href="/wiki/Archivo:Bandera_provincia_Albacete.svg"><img class="mw-file-element" data-file-height="500" data-file-width="750" decoding="async" height="13" src="//upload.wikimedia.org/wikipedia/commons/thumb/d/d6/Bandera_provincia_Albacete.svg/20px-Bandera_provincia_Albacete.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/d/d6/Bandera_provincia_Albacete.svg/40px-Bandera_provincia_Albacete.svg.png 1.5x" width="20"/></a>,
 <a href="/wiki/Provincia_de_Albacete" title=

In [19]:
enlaces_df = pd.DataFrame({
    'provincia': [enlace.get('title', enlace.text.strip()) for enlace in enlaces],
    'enlace': [enlace.get('href', '') for enlace in enlaces]
})

In [20]:
enlaces_df

,provincia,enlace
0,,/wiki/Archivo:Flag_of_%C3%81lava.svg
1,Álava,/wiki/%C3%81lava
2,,/wiki/Archivo:Bandera_provincia_Albacete.svg
3,Provincia de Albacete,/wiki/Provincia_de_Albacete
4,Provincia de Alicante,/wiki/Provincia_de_Alicante
...,...,...
86,,/wiki/Archivo:Bandera_de_Vizcaya.svg
87,Vizcaya,/wiki/Vizcaya
88,Provincia de Zamora,/wiki/Provincia_de_Zamora
89,,/wiki/Archivo:Flag_of_Zaragoza_province_(with_...


Por ejemplo, creamos la sopa para la primera provincia:

In [36]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
url_provincias = "https://es.wikipedia.org/wiki/Provincia_(España)"
respuesta_provincias = requests.get(url_provincias, headers=headers)
respuesta_provincias.raise_for_status()

In [37]:
r=requests.get("https://es.wikipedia.org"+enlaces_df.loc[0, 'enlace'], headers=headers)
sopa_provincia=BeautifulSoup(r.text, "html.parser")

Vamos a extraer en un data frame la información geográfica de la tabla de la barra lateral derecha (atributo de clase `infobox`):

In [40]:
tabla = sopa_provincia.find("table", class_="infobox")
if tabla:
    print("Tabla encontrada correctamente")
else:
    print("No se encontró la tabla infobox")

No se encontró la tabla infobox


Si inspeccionas su estructura HTML verás una serie de tags `tr` de las que cuelgan pares de tags `th` y `td` asociadas. Capturaremos sus textos en dos listas: `dato` y `valor`, respectivamante.

In [39]:
dato, valor = [], []
for t in tabla.find_all("tr"):
    if t.th and t.td:
        dato.append(t.th.text.strip())
        valor.append(t.td.text.strip())

AttributeError: 'NoneType' object has no attribute 'find_all'

In [ ]:
datos = pd.DataFrame({
    'Dato': dato,
    'Valor': valor
})
datos

,Dato,Valor
0,Coordenadas,"42°50′00″N 2°45′00″O ﻿ / ﻿ 42.833333333333, -2.75"
1,Capital,Vitoria
2,Idioma oficial,Español y euskera
3,Entidad,Provincia de España
4,• País,España España
5,• Comunidad,País Vasco País Vasco
6,CongresoSenadoParlamento VascoJuntas Generales...,4 diputados 4 senadores 25 parlamentarios auto...
7,Subdivisiones,7 comarcas 51 municipios
8,Fundación,División territorial de 1833
9,Superficie,Puesto 48.º


### Ejercicio 3
Crea una tabla (dataframe) con la capital, la superficie y la población de cada provincia de España.\
Para encontrar en la tabla estos datos podemos hacer:

'\nVitoria'

'\xa0• Total\n3037 km²\xa0(0,60\xa0%)'

'\xa0• Total\n334\xa0412\xa0hab.\xa0(0,70\xa0%)'

Tendrás que usar expresiones regulares para extraer de estos strings el texto buscado.

                              Provincia  \
0                                 Álava   
1                 Provincia de Albacete   
2                 Provincia de Alicante   
3                  Provincia de Almería   
4                              Asturias   
5                    Provincia de Ávila   
6                  Provincia de Badajoz   
7                Provincia de Barcelona   
8                   Provincia de Burgos   
9                  Provincia de Cáceres   
10                   Provincia de Cádiz   
11                            Cantabria   
12               Provincia de Castellón   
13             Provincia de Ciudad Real   
14        Provincia de Córdoba (España)   
15               Provincia de La Coruña   
16                  Provincia de Cuenca   
17                  Provincia de Gerona   
18                 Provincia de Granada   
19             Provincia de Guadalajara   
20                            Guipúzcoa   
21                  Provincia de Huelva   
22         